In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
%matplotlib inline

data = pd.read_csv('C:\\Users\\Owner\\Napa\\results_model_data.csv')

In [2]:
def result_assign(win_margin):
    # This function converts the win_margin column into a binary win/loss result
    if win_margin>0:
        return 1
    else:
        return 0

In [3]:
def sigmoid(z):
    # Computes the sigmoid function for logistic regression
    return 1 / (1 + np.exp(-z))

In [4]:
def sigmoid_gradient(z):
    # Computes the gradient of the sigmoid function, to be used in backpropagation
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [5]:
def forward_propagate(X, theta1, theta2):
    # Calculate the hypothesis using input values of theta for each stage of the network
    m = X.shape[0]
    # Insert bias unit for input layer
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)   
    z2 = a1 * theta1.T
    # Insert bias unit for hidden layer
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h

In [6]:
def backward_prop(params, input_layer_size, hidden_layer_size, num_labels, X, y):

    # Reshape the parameter array back into the respective matrices
    theta1 = np.matrix(np.reshape(params[:hidden_layer_size * (input_layer_size + 1)], (hidden_layer_size, (input_layer_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_layer_size * (input_layer_size + 1):], (num_labels, (hidden_layer_size + 1))))
    
    # Forward propagate through the network
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # Initialize
    J = 0
    delta1 = np.zeros(theta1.shape)
    delta2 = np.zeros(theta2.shape)
    
    # Compute cost
    first = np.multiply(-y, np.log(h))
    second = np.multiply((1 - y), np.log(1 - h))
    J = np.sum(first - second) / m
    
    # Backpropagate to get gradients   
    d3 = h - y
    d2 = np.multiply((d3*theta2[:,1:hidden_layer_size+1]), sigmoid_gradient(z2))  
    delta1 = (np.matmul(a1.T, d2)).T / m
    delta2 = (np.matmul(d3.T, a2)) / m
    
    # Reshape gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [7]:
# Add a new binary column to the data, which has value 1 where the result is positive, and 0 if negative
data['Result'] = data.apply(lambda x: result_assign(x['Win Margin']),axis=1)
# Select only quantitive paramaters to be used in the model
model_data = data[['Race Margin', 'Win % Margin', 'Skill Margin', 'Game Margin', 'AvgPPM Margin', 'Result']]
model_data.head()

,Race Margin,Win % Margin,Skill Margin,Game Margin,AvgPPM Margin,Result
0,2.0,3.750000,33,0,1.90,1
1,1.0,-11.805556,14,0,-1.19,1
2,0.0,-9.583333,7,0,-0.30,1
3,-2.0,10.000000,-10,0,2.72,1
4,1.0,14.545455,18,0,3.48,1


In [8]:
# Set X (training data) and y (target variable)
cols = model_data.shape[1]
X = model_data.iloc[:,0:cols-1]
y = model_data.iloc[:,cols-1:cols]
y0 = y
# Split the data into training and validation sets with 80/20 ratio
train_X, val_X, train_y, val_y = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state = 0)

# Convert to numpy matrices
m = X.shape[0]
X_train = np.matrix(train_X)
y_train = np.matrix(train_y)
X_val = np.matrix(val_X)
y_val = np.matrix(val_y)

# Define architecture of neural network
input_layer_size  = cols-1;  # Each match has 5 features
hidden_layer_size = 50;      # 50 hidden units
num_labels = 1;              # Win/Loss parameter

# Randomly initialize the input parameter array, with values normalized by length
epsilon_1 = np.sqrt(6./(hidden_layer_size + input_layer_size))
epsilon_2 = np.sqrt(6./(hidden_layer_size + num_labels))
param1 = np.random.random(size=hidden_layer_size * (input_layer_size + 1))*2*epsilon_1 - epsilon_1
param2 = np.random.random(size=num_labels * (hidden_layer_size + 1))*2*epsilon_2 - epsilon_2
params = np.concatenate((param1,param2))

In [9]:
# Minimize the backpropagation cost function
fmin = minimize(fun=backward_prop, x0=params, args=(input_layer_size, hidden_layer_size, num_labels, X_train, y_train), 
                method='TNC', jac=True, options={'maxiter': 250})

# Retrieve the corresponding theta parameters and reshape to matrices
theta1 = np.matrix(np.reshape(fmin.x[:hidden_layer_size * (input_layer_size + 1)], (hidden_layer_size, (input_layer_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_layer_size * (input_layer_size + 1):], (num_labels, (hidden_layer_size + 1))))

# Calculate predictions based on the model
a1_t, z2_t, a2_t, z3_t, h_t = forward_propagate(X_train, theta1, theta2)
a1_v, z2_v, a2_v, z3_v, h_v = forward_propagate(X_val, theta1, theta2)
y_pred_train = [1 if i>=0.5 else 0 for i in h_t]
y_pred_val = [1 if i>=0.5 else 0 for i in h_v]

# Compare predictions to actual data
correct_train = [1 if a == b else 0 for (a, b) in zip(y_pred_train, y_train)]
correct_val = [1 if a == b else 0 for (a, b) in zip(y_pred_val, y_val)]
accuracy_train = (sum(map(int, correct_train)) / float(len(correct_train)))
accuracy_val = (sum(map(int, correct_val)) / float(len(correct_val)))
print 'Train accuracy = {0}%'.format(accuracy_train * 100)
print 'Validation accuracy = {0}%'.format(accuracy_val * 100)

C:\Users\Owner\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


Train accuracy = 63.0615453729%
Validation accuracy = 63.3123689727%
